In [ ]:
# bash code to load data into BQ
# bq load --source_format=CSV crypto.crypto_data gs://anant-caserta-eval/crypto_data.csv \
# available_supply:FLOAT,name:STRING,percent_change_1h:FLOAT,percent_change_24h:FLOAT,last_updated:FLOAT,market_cap_usd:FLOAT,symbol:STRING,\
# volume_24h_usd:FLOAT,price_usd:FLOAT,percent_change_7d:FLOAT,total_supply:FLOAT,id:STRING,price_btc:FLOAT,rank:FLOAT,max_supply:FLOAT

In [3]:
import time
import six
try:
    import pandas
except ImportError:
    pandas = None

from google.cloud import bigquery

In [31]:
client = bigquery.Client()
def run_query(query):
  query_job = client.query(query)
  return list(query_job.result())

In [32]:

#coins with price > $8k
query_8k = 'with price_8k as (select count(*) as count_8k from `crypto.crypto_data`  where price_usd > 8000) select * from price_8k' 
print('Number of currencies with count more that $ 8000: {}'.format(run_query(query_8k)[0][0]))

Number of currencies with count more that $ 8000: 1


In [33]:
query_marketcap = '''with total_marketcap as 
(select market_cap_usd as market_cap from `crypto.crypto_data`  
order by market_cap_usd DESC LIMIT 100) select sum(market_cap) 
as sum_marketcap from total_marketcap'''
print('Total market cap of the top 100 currencies is : $ {:10.2f}'.format(run_query(query_marketcap)[0][0]))

Total market cap of the top 100 currencies is : $ 411094574210.00


In [34]:
query_marketcap_5m = '''with supply_lt5k as (select name from `crypto.crypto_data`  where available_supply < 5000000) select name from supply_lt5k'''
print('Currencies with market cap less than 5M: {}'.format([item[0] for item in run_query(query_marketcap_5m)]))

Currencies with market cap less than 5M: [u'Byteball Bytes', u'Mixin', u'Maker', u'DigixDAO', u'Zcash', u'Veritaseum', u'ZCoin']


In [35]:
query_greatest_growth = '''with greatest_growth as (select name from `crypto.crypto_data`  ORDER BY percent_change_7d DESC LIMIT 5 ) select name from greatest_growth'''
print('Currencies with greatest growth: {}'.format([item[0] for item in run_query(query_greatest_growth)]))

Currencies with greatest growth: [u'Mixin', u'Bitcoin Private', u'Bitcoin Cash', u'Dentacoin', u'Storm']


In [36]:
contains_x = '''with contains_x as (select name from `crypto.crypto_data`  where name like "%X%" ) select name from contains_x'''
print('Currencies with greatest growth: {}'.format([item[0] for item in run_query(contains_x)]))

Currencies with greatest growth: [u'PIVX', u'TenX', u'WAX', u'Pundi X', u'GXChain']
